## Load Library

In [1]:
# Epoch결과를 CSV파일로 streaming : CSVLogger
# 각 Epoch 후 Model을 저장 : ModelCheckpoint
# 개선되지 않으면 종료 : EarlyStopping
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
# 학습이 정체되면 Learning Rate 감소
from keras.callbacks import ReduceLROnPlateau
# 실시간 데이터 증가로 텐서 이미지의 batch 생성. Data augmentation 시 활용
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
# GAP(Global Average Pooling)은 그냥 Pooling과 다른 용도로 사용.
# 2D -> 1D로 만드는 효과. 이로 인해 최종 출력에 FC 대신에 사용할 수 있다.
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
# Depthwise 진행 후 Separable. Depthwise Separable Convolution
# Conv 진행, 서로 다른 channel끼리 정보 공유, parameter 감소 가능
from keras.layers import SeparableConv2D
from keras import layers
# L1(절댓값)은 0에서 미분 불가능하며, outlier에 민감하지 않다.
# 그래서 L2 regularization을 사용.
from keras.regularizers import l2

import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
import os
import glob

## Load Datasets

In [2]:
# 200x200 size image

images = []
target = []
idx = 0

data_folder = 'data'
folder_list = os.listdir(data_folder)

for folder in folder_list:
    folder = data_folder + '/' + folder
    print(folder)
    files = glob.glob(os.path.join(folder,"*.jpg"))
    print(files)
    for filename in files:
        temp = cv2.imread(filename, 0)
        images.append(temp)
        target.append(idx)
    idx += 1

data/neutral
['data/neutral\\user1.jpg', 'data/neutral\\user10.jpg', 'data/neutral\\user100.jpg', 'data/neutral\\user1000.jpg', 'data/neutral\\user101.jpg', 'data/neutral\\user102.jpg', 'data/neutral\\user103.jpg', 'data/neutral\\user104.jpg', 'data/neutral\\user105.jpg', 'data/neutral\\user106.jpg', 'data/neutral\\user107.jpg', 'data/neutral\\user108.jpg', 'data/neutral\\user109.jpg', 'data/neutral\\user11.jpg', 'data/neutral\\user110.jpg', 'data/neutral\\user111.jpg', 'data/neutral\\user112.jpg', 'data/neutral\\user113.jpg', 'data/neutral\\user114.jpg', 'data/neutral\\user115.jpg', 'data/neutral\\user116.jpg', 'data/neutral\\user117.jpg', 'data/neutral\\user118.jpg', 'data/neutral\\user119.jpg', 'data/neutral\\user12.jpg', 'data/neutral\\user120.jpg', 'data/neutral\\user121.jpg', 'data/neutral\\user122.jpg', 'data/neutral\\user123.jpg', 'data/neutral\\user124.jpg', 'data/neutral\\user125.jpg', 'data/neutral\\user126.jpg', 'data/neutral\\user127.jpg', 'data/neutral\\user128.jpg', 'dat

data/smile
['data/smile\\user1.jpg', 'data/smile\\user10.jpg', 'data/smile\\user100.jpg', 'data/smile\\user1000.jpg', 'data/smile\\user101.jpg', 'data/smile\\user102.jpg', 'data/smile\\user103.jpg', 'data/smile\\user104.jpg', 'data/smile\\user105.jpg', 'data/smile\\user106.jpg', 'data/smile\\user107.jpg', 'data/smile\\user108.jpg', 'data/smile\\user109.jpg', 'data/smile\\user11.jpg', 'data/smile\\user110.jpg', 'data/smile\\user111.jpg', 'data/smile\\user112.jpg', 'data/smile\\user113.jpg', 'data/smile\\user114.jpg', 'data/smile\\user115.jpg', 'data/smile\\user116.jpg', 'data/smile\\user117.jpg', 'data/smile\\user118.jpg', 'data/smile\\user119.jpg', 'data/smile\\user12.jpg', 'data/smile\\user120.jpg', 'data/smile\\user121.jpg', 'data/smile\\user122.jpg', 'data/smile\\user123.jpg', 'data/smile\\user124.jpg', 'data/smile\\user125.jpg', 'data/smile\\user126.jpg', 'data/smile\\user127.jpg', 'data/smile\\user128.jpg', 'data/smile\\user129.jpg', 'data/smile\\user13.jpg', 'data/smile\\user130.

data/_emotion_training.log
[]


## Train/Test Split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(images, target, test_size = 0.2, random_state = 42)

In [41]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

- Grayscale

In [42]:
X_train = X_train.reshape(-1, 200, 200, 1)
X_test = X_test.reshape(-1, 200, 200, 1)

In [43]:
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [44]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1600, 200, 200, 1) (400, 200, 200, 1) (1600,) (400,)


## Preprocessing

- Scaling

In [45]:
X_train = X_train / 255.0
X_test = X_test / 255.0

# Model

## Parameters

- num_classes 로 target(Emotion) 개수 조정

In [46]:
# sharp local minima에 빠지지 않기 위해.
batch_size = 32 # 보다 정확한 값을 원하면 16, 8까지는 줄일 수 있다. 예전에 읽은 논문에 따르면 8보다 작으면 오히려 성능이 감소할 수 있다.
num_epochs = 110
input_shape = (48, 48, 1)
verbose = 1 # 상세한 logging을 출력하겠다는 의미
num_classes = 1 # 결과물 조정
patience = 50
base_path = 'data/'
l2_regularization = 0.01 # L2 정규화 사용

### Datagen

In [47]:
# data generator 일단 생략
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

## Model paramters

In [48]:
# model parameters
regularization = l2(l2_regularization)

In [ ]:
'''
# base
img_input = Input(input_shape)
# use_bias(편향 사용) = False로 하는 이유 : batchnormalization을 사용하는 경우 이후에 
# 평균에서 더해진 값(편향)이 빠지기 때문에 편향을 사용하는 효과가 없어지므로 False
x = Conv2D(8, (3, 3), strides = (1, 1), kernel_regularizer = regularization, use_bias = False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides = (1, 1), kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# module 1
residual = Conv2D(16, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])

# module 2
residual = Conv2D(32, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])

# module 3
residual = Conv2D(64, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])

# module 3
residual = Conv2D(64, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])

# module 4
residual = Conv2D(128, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding = 'same', kernel_regularizer = regularization, use_bias = False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding = 'same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax', name = 'predictions')(x)

model = Model(img_input, output)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()
'''

In [53]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(input_shape=(200, 200,1), kernel_size=(3,3), filters=32, padding='same', activation='relu'),
  tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  tf.keras.layers.Dropout(rate=0.5),
  tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding='same', activation='relu'),
  tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=256, padding='valid', activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  tf.keras.layers.Dropout(rate=0.5),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=512, activation='relu'),
  tf.keras.layers.Dropout(rate=0.5),
  tf.keras.layers.Dense(units=256, activation='relu'),
  tf.keras.layers.Dropout(rate=0.5),
  tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 200, 200, 32)      320       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 200, 200, 64)      18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 100, 100, 64)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 100, 64)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 100, 100, 128)     73856     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 98, 98, 256)       295168    
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 49, 49, 256)      

In [ ]:
model.fit(X_train, y_train, epochs = 25, validation_split = 0.25)

Epoch 1/25
38/38 [==============================] - 313s 8s/step - loss: 2.3197 - accuracy: 0.5067 - val_loss: 0.7217 - val_accuracy: 0.7625
Epoch 2/25
38/38 [==============================] - 377s 10s/step - loss: 0.7212 - accuracy: 0.5517 - val_loss: 0.6564 - val_accuracy: 0.5775
Epoch 3/25
38/38 [==============================] - 390s 10s/step - loss: 0.5705 - accuracy: 0.7025 - val_loss: 0.2761 - val_accuracy: 0.9850
Epoch 4/25
38/38 [==============================] - 391s 10s/step - loss: 0.3054 - accuracy: 0.8933 - val_loss: 0.0947 - val_accuracy: 0.9650
Epoch 5/25
38/38 [==============================] - 392s 10s/step - loss: 0.0749 - accuracy: 0.9767 - val_loss: 0.0250 - val_accuracy: 0.9975
Epoch 6/25
38/38 [==============================] - 389s 10s/step - loss: 0.0303 - accuracy: 0.9892 - val_loss: 3.6003e-04 - val_accuracy: 1.0000
Epoch 7/25
38/38 [==============================] - 390s 10s/step - loss: 0.0214 - accuracy: 0.9925 - val_loss: 2.1437e-04 - val_accuracy: 1.0000

## Callbacks

In [ ]:
'''
# log_file_path = base_path + '_emotion_training.log'
# csv_logger = CSVLogger(log_file_path, append = False)
early_stop = EarlyStopping('val_loss', patience = patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor = 0.1, patience = int(patience/4), verbose = 1)
trained_models_path = base_path + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose = 1, save_best_only = True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

model.fit_generator(data_generator.flow(X_train, y_train, batch_size),
                   steps_per_epoch = len(X_train) / batch_size,
                   epochs = num_epochs, verbose = 1, callbacks = callbacks,
                   validation_data = (X_test, y_test))
model.fit()
'''

# 평가

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label = 'loss')
plt.plot(history.history['val_loss'], 'r--', label = 'val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label = 'accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()

model.evaluate(X_test, y_test)